# Imports

In [1]:
import boto3
import sagemaker
from sagemaker import Session
from sagemaker.session import TrainingInput
from sagemaker.tensorflow import TensorFlow as tf
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter, CategoricalParameter
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import tensorflow
import s3fs
import numpy as np
import json
import time
RANDOM_SEED = 0
sagemaker.__version__

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


2025-08-16 04:11:42.602547: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


'2.245.0'

# Sagemaker Initialization

In [20]:
fs = s3fs.S3FileSystem()
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
s3 = boto3.client("s3")
bucket = sess.default_bucket()   # will be sagemaker-ap-southeast-2-838084669510
prefix = "aiornot"
print(f"S3 Bucket: f{bucket}")

s3_train_path = f"s3://{bucket}/{prefix}/train/train.npz"
s3_small_train_path = f"s3://{bucket}/{prefix}/small_train/small_train.npz"

s3_test_path = f"s3://{bucket}/{prefix}/test/test.npz"
s3_holdout_test_path = f"s3://{bucket}/{prefix}/holdout_test/holdout_test.npz"
s3_output_location = f"s3://{bucket}/{prefix}/model_output"

train_input = TrainingInput(s3_train_path, content_type="application/x-npz")
test_input = TrainingInput(s3_test_path, content_type="application/x-npz")

S3 Bucket: fsagemaker-ap-southeast-2-838084669510


# Transfer Learning Configuration

In [ ]:
metric_definitions = [
    {"Name": "val_auc",       "Regex": "val_auc: ([0-9\\.]+)"},
    {"Name": "val_f1",        "Regex": "val_f1: ([0-9\\.]+)"},
    {"Name": "val_precision", "Regex": "val_precision: ([0-9\\.]+)"},
    {"Name": "val_recall",    "Regex": "val_recall: ([0-9\\.]+)"},
    {"Name": "val_accuracy",  "Regex": "val_accuracy: ([0-9\\.]+)"},
]

estimator = tf(
    entry_point="train_transfer.py",
    source_dir="src",
    role=role,
    instance_type="ml.c5.2xlarge",
    instance_count=1,
    framework_version="2.14",
    py_version="py310",
    output_path=f"s3://{bucket}/model_output",
    hyperparameters={
        "epochs": 3,
        "height": 224,
        "width": 224,
        "channels": 3,
        "batch-size": 1,
        "learning-rate": 5e-5, # snall to allow for fine tuning
        "dropout-rate": 0.2,
        "unfreeze-fraction": 0.10,
    },
    metric_definitions=metric_definitions,
)

job_name = f"transfer-learning-{time.strftime('%Y%m%d-%H%M%S')}"
estimator.fit({"train": train_input, "test": test_input}, job_name=job_name)

note: see final model deployment in `model_evaluation.ipynb`